<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 2 ( Recuperación de Información) 

## Profesora: Lisibonny Beato
### Período 3-2023-2024</div>


In [2]:
# Importando algunas librerias

import pandas as pd
import numpy as np
from IPython.core.release import author
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from os import path
from textblob import TextBlob
import xml.etree.ElementTree as ET


<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>El propósito de esta asignación es que el estudiante ponga en práctica la construcción de sistemas de recuperación de información basado en el modelo clásico de RI Vector Space Model, así como también que evalue la efectividad de estos modelos mediante el uso de una colección de referencia (Benchmark).
<br />
<br />
Para esta práctica estará utilizando el siguiente repositorio:https://github.com/oussbenk/cranfield-trec-dataset. En el mismo encontrarán los archivos  cran.all.1400.xml, cran.qry.xml, cranqrel.trec.txt:
<ul>
<li>cran.all.1400.xml contiene 1,400 resumenes de artículos científicos.</li>
<li>cran.qry.xml 225 términos que representan consultas.</li>
<li>cranqrel.trec.txt contiene los juicios de relevancia a dichas consultas.</li>
    </ul>  
<br />
<br />
Estudie en detalle la estructura y el contenido de este conjunto de documentos provistos antes de comenzar.    
<br />
<br />
En este trabajo, aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


## 1. Ejercicio 1
### Puntuación máxima de la tarea: 3 puntos
#### Limpieza y preparación de los datos, utilizando distintas técnicas de las ya vistas en clases. Para esta tarea utilizará el archivo cran.all.1400.xml, específicamente sus columnas title y text.


In [3]:
def parse_cranfield_xml(file_path):
    """
    Parse Cranfield XML file by handling the specific format
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # The file likely has multiple <DOC> elements without a root wrapper
        # Let's wrap the entire content in a root element
        wrapped_content = f'<root>\n{content}\n</root>'

        # Try to parse the wrapped content
        root = ET.fromstring(wrapped_content)
        return root

    except ET.ParseError as e:
        print(f"ParseError: {e}")
        # If that fails, let's try a different approach
        return parse_cranfield_alternative(file_path)

In [4]:
def parse_cranfield_alternative(file_path):
    """
    Alternative parsing method using regex to extract documents
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Find all DOC elements using regex
    doc_pattern = r'<DOC>(.*?)</DOC>'
    doc_matches = re.findall(doc_pattern, content, re.DOTALL)

    documents = []
    for i, doc_content in enumerate(doc_matches):
        try:
            # Create a proper XML structure for each document
            wrapped_doc = f'<DOC>{doc_content}</DOC>'
            doc_element = ET.fromstring(wrapped_doc)
            documents.append(doc_element)
        except ET.ParseError as e:
            print(f"Error parsing document {i}: {e}")
            continue

    return documents

In [5]:
# Parse the XML file
try:
    root = parse_cranfield_xml('cranfield-trec-dataset/cran.all.1400.xml')

    # Extract data based on the structure we find
    data = []

    if isinstance(root, list):
        # If we got a list from the alternative parser
        docs = root
    else:
        # If we got an ElementTree root
        docs = root.findall('.//DOC')

    print(f"Found {len(docs)} documents")

    for doc in docs:
        docno = doc.find('DOCNO').text.strip() if doc.find(
            'DOCNO') is not None else ''
        title = doc.find('TITLE').text.strip() if doc.find(
            'TITLE') is not None else ''
        author_elem = doc.find('AUTHOR')
        author = author_elem.text.strip() if author_elem is not None and author_elem.text else ''
        bib_elem = doc.find('BIB')
        bib = bib_elem.text.strip() if bib_elem is not None and bib_elem.text else ''
        text_elem = doc.find('TEXT')
        text = text_elem.text.strip() if text_elem is not None and text_elem.text else ''

        data.append({
            'docno': docno,
            'title': title,
            'author': author,
            'bib': bib,
            'text': text
        })

    # Convert the data to a DataFrame
    df = pd.DataFrame(data)
    print(f"Created DataFrame with {len(df)} rows")
    print(f"Columns: {df.columns.tolist()}")

    # Show first few rows to verify
    print("\nFirst 5 rows:")
    print(df[['docno', 'title']].head())

except Exception as e:
    print(f"Error: {e}")
    print("Let's examine the file structure more carefully...")

    # Fallback: examine the file structure
    with open('cranfield-trec-dataset/cran.all.1400.xml', 'r',
              encoding='utf-8') as f:
        lines = f.readlines()

    print(f"File has {len(lines)} lines")
    print("First 20 lines:")
    for i, line in enumerate(lines[:20]):
        print(f"{i + 1:2d}: {line.rstrip()}")

Found 0 documents
Created DataFrame with 0 rows
Columns: []

First 5 rows:
Error: "None of [Index(['docno', 'title'], dtype='object')] are in the [columns]"
Let's examine the file structure more carefully...
File has 36734 lines
First 20 lines:
 1: <doc>
 2: <docno>1</docno>
 3: <title>experimental investigation of the aerodynamics of a
 4: wing in a slipstream .</title>
 5: <author>brenckman,m.</author>
 6: <bib>j. ae. scs. 25, 1958, 324.</bib>
 7: <text>experimental investigation of the aerodynamics of a
 8: wing in a slipstream .
 9:   an experimental study of a wing in a propeller slipstream was
10: made in order to determine the spanwise distribution of the lift
11: increase due to slipstream at different angles of attack of the wing
12: and at different free stream to slipstream velocity ratios .  the
13: results were intended in part as an evaluation basis for different
14: theoretical treatments of this problem .
15:   the comparative span loading curves, together with
16: supp

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

In [ ]:
%%sql


## 2. Ejercicio 2
### Puntuación máxima de la tarea: 2 puntos
#### Construir al menos dos modelos de recuperación de información basado en el modelo vectorial, visto en detalle en clases, cada uno con una configuración distinta de textos, tal y como se indica a continuación: solo title y title+text.

## 3. Ejercicio 3
### Puntuación máxima de la tarea: 5 puntos
#### Evaluar los modelos de recuperación de información construidos, mediante la  utilización de la colección de referencia dada (Archivos cran.qry.xml y cranqrel.trec.txt  ).  Seleccione dos métricas ampliamente utilizadas en la evaluación de este tipo de sistemas y analice los resultados para determinar cual es el mejor de los modelos. 